In [1]:
from functions import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
output_path = Path("../modular_polygon_generation/libcore/data/maps/")
pcd = load_point_cloud(pcd_path)
print(f"Number of points in original point cloud: {len(pcd.points)}")

area_1_downsampled_0_5 = pcd.voxel_down_sample(voxel_size=0.5)
print(f"Number of points in downsampled point cloud (0.5): {len(area_1_downsampled_0_5.points)}")
visualize(VisObjPCD(area_1_downsampled_0_5))
o3d.io.write_point_cloud(str(output_path / "area_1_downsampled_0_5.pcd"), area_1_downsampled_0_5, write_ascii=False)

area_1_downsampled_0_2 = pcd.voxel_down_sample(voxel_size=0.2)
print(f"Number of points in downsampled point cloud (0.2): {len(area_1_downsampled_0_2.points)}")
visualize(VisObjPCD(area_1_downsampled_0_2))
o3d.io.write_point_cloud(str(output_path / "area_1_downsampled_0_2.pcd"), area_1_downsampled_0_2, write_ascii=False)

area_1_downsampled_0_1 = pcd.voxel_down_sample(voxel_size=0.1)
print(f"Number of points in downsampled point cloud (0.1): {len(area_1_downsampled_0_1.points)}")
visualize(VisObjPCD(area_1_downsampled_0_1))
o3d.io.write_point_cloud(str(output_path / "area_1_downsampled_0_1.pcd"), area_1_downsampled_0_1, write_ascii=False)

area_1_downsampled_0_05 = pcd.voxel_down_sample(voxel_size=0.05)
print(f"Number of points in downsampled point cloud (0.05): {len(area_1_downsampled_0_05.points)}")
visualize(VisObjPCD(area_1_downsampled_0_05))
o3d.io.write_point_cloud(str(output_path / "area_1_downsampled_0_05.pcd"), area_1_downsampled_0_05, write_ascii=False)

area_1_downsampled_0_01 = pcd.voxel_down_sample(voxel_size=0.01)
print(f"Number of points in downsampled point cloud (0.01): {len(area_1_downsampled_0_01.points)}")
visualize(VisObjPCD(area_1_downsampled_0_01))
o3d.io.write_point_cloud(str(output_path / "area_1_downsampled_0_01.pcd"), area_1_downsampled_0_01, write_ascii=False)



#visualize(VisObjPCD(area_1_downsampled_0_01))
#visualize(VisObjPCD(area_1_downsampled_0_05))
#visualize(VisObjPCD(area_1_downsampled_0_1))
#visualize(VisObjPCD(area_1_downsampled_0_2))

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...


KeyboardInterrupt: 

In [3]:
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/run_1756996692")
output_path = Path("../MSSP/input/")
mssp_output_path = Path("../MSSP/output/")

chen_skeleton_ids = np.load(mssp_output_path / "chen_ske_ids.npy")
chen_skeleton_dist = np.load(mssp_output_path / "chen_ske_dist.npy")

# Load point cloud
pcd = load_point_cloud(pcd_path)

# rasterize point cloud
voxel_grid, transformation_matrix = rasterize_point_cloud(pcd, voxel_size=0.1)

# fill holes
voxel_grid_flooded = flood_voxel_grid(voxel_grid, direction='up')
voxel_grid_flooded = flood_voxel_grid(voxel_grid_flooded, direction='down')
# invert it to get the free space
voxel_grid_inverted = invert_voxel_grid(voxel_grid_flooded)

# skeletonize
#skeleton = skeletonize_voxel_grid(voxel_grid_flooded, dilation_size=2)

# Load chen skeleton graph
chen_skeleton_graph = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")

visualize([VisObjGraph(chen_skeleton_graph), VisObjVoxel(voxel_grid_flooded, transformation_matrix), VisObjPCD(pcd)])

chen_skeleton_voxel, chen_skeleton_id_map = get_voxel_from_graph(chen_skeleton_graph, transformation_matrix, voxel_grid_flooded.shape)
visualize([VisObjVoxel(chen_skeleton_voxel, transformation_matrix), VisObjPCD(pcd)])

chen_visSkelDist = VisObjVoxel(chen_skeleton_dist, np.load(output_path / "M.npy"), noise=True)
visualize(chen_visSkelDist)

chen_visSkelIds = VisObjVoxel(chen_skeleton_ids, np.load(output_path / "M.npy"), noise=True)
visualize(chen_visSkelIds)

M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    if voxel_grid_inverted[tuple(v_pos_1.astype(int))] == 0:
        outlaiers.add(edge[0])
        print(f"Node {n1} maps to free space voxel {v_pos_1}")
    if voxel_grid_inverted[tuple(v_pos_2.astype(int))] == 0:
        outlaiers.add(edge[1])
        print(f"Node {n2} maps to free space voxel {v_pos_2}")

print(f"Total outlier nodes: {len(outlaiers)}")

visualize([VisObjGraph(chen_skeleton_graph, outlaiers), VisObjVoxel(voxel_grid_flooded, transformation_matrix), VisObjPCD(pcd)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Rasterizing point cloud with voxel size 0.1...
Voxel grid created with shape (248, 484, 54)
Filling voxel grid from direction up...
Voxel grid filled from direction up.
Filling voxel grid from direction down...
Voxel grid filled from direction down.
Inverting voxel grid...
Voxel grid inverted.
Loading graph from ../output/run_1756996692/node_list.txt and ../output/run_1756996692/edge_list.txt...
Loaded graph with 891 nodes and 1043 edges.
Converting graph to voxel grid...
Graph converted to voxel grid.
Node {'coords': array([-8.96824,  2.36996,  1.38798])} maps to free space voxel [137.04759191  46.36960008  22.10980002]
Node {'coords': array([-10.5268 ,   3.13996,   1.42364])} maps to free space voxel [121.46199191  54.06960008  22.46640002]
Node {'coords': array([-10.5268 ,   3.13996,   1.42364]

In [ ]:
(chen_skeleton_voxel & voxel_grid_flooded).sum()

np.int64(14)

In [13]:
from skimage.draw import line_nd
# inverse transformation matrix
M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    # compute line voxels between v_pos_1 and v_pos_2
    line_voxels = np.array(line_nd(tuple(v_pos_1.astype(int)), tuple(v_pos_2.astype(int)))).T[1:] # skip the first entry, hence its the node itself
    for voxel in line_voxels:
        if voxel_grid[tuple(voxel)] == 1:
            outlaiers.add(edge)
print(f"Total outlier nodes: {len(outlaiers)}")

Total outlier nodes: 10


In [12]:
from skimage.draw import line_nd
for voxel in np.array(line_nd((0,0,0), (3,4,5))).T[1:]:
    print(voxel)

[1 1 1]
[1 2 2]
[2 2 3]
[2 3 4]
